In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")

# Index incidents using Incident ID
features = ['Incident ID', 'Date', 'State', 'City/County', 'Address', 'Nr Killed', 'Nr Injured',
           'Incident URL', 'Source URL', 'Incident URL Fields Missing',
           'Congressional District', 'Gun Stolen', 'Gun Type',
           'Incident Characteristics', 'Latitude', 'Location Description',
           'Longitude', 'Nr Guns Involved', 'Notes', 'Participant Age',
           'Participant Age Group', 'Participant Gender', 'Participant Name',
           'Participant Relationship', 'Participant Status', 'Participant Type',
           'Sources', 'State House District', 'State Senate District']
data.columns = features
data.set_index("Incident ID", inplace=True)